####Load Google Drive File

In [33]:
## Load Folder Drive
import os, sys
import os.path
from os import path
from google.colab import drive
drive.mount('/content/drive/')

## Move Pointer File ke DATA_UM
%cd /content/drive/MyDrive/DATA_UM

## Declare Root Project File
root_file = "/content/drive/MyDrive/DATA_UM"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/DATA_UM


# **SECTION FOR DATA SCRAPING**

In [34]:
import pandas as pd
import json
import csv

# dframe = pd.read_csv('data_scraped/'+hashtag+'.csv')

# **SECTION FOR DATA CLEANING**

####1. Data Transformation (Old Process) - Updated Data Transformasi di Collab yg berbeda di 1.2 beda tiap platform. Disini hanya tinggal load. (Do not delete cell)

In [35]:
import re

Load Data Twitter dan Tiktok sisanya menyusul.

In [36]:
df_clean = pd.read_excel(open('data/label/RTP_FINAL_TIKTOK.xlsx', 'rb'),sheet_name='dataset')
df_clean.reset_index(drop=True, inplace=True)
df_clean = df_clean.loc[:, ~df_clean.columns.str.contains('^Unnamed')]
df_clean

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,29-11-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,황 펠리/황 휴즈,hwang_felly,https://www.tiktok.com/@hwang_felly,0,NaN,NaN,NaN,2,https://www.tiktok.com/@tedenn/video/730678176...,protes ke ultra udh excited padahal,1
20047,15-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,youareperfect,bantufypdon,https://www.tiktok.com/@bantufypdon,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,αe`oO ✧: tostos nct aja blm dapet😭😭,1
20048,1-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,📧,bbabiela,https://www.tiktok.com/@bbabiela,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,MKSDDD?!,1
20049,3-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,Sionah 🌟,sionahla,https://www.tiktok.com/@sionahla,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,tolong Kids,1


####2. Text Transformation
1. Delete Punctuations,
2. Delete Number,
3. Delete Link Website,
4. Delete Hashtag,
5. Transform to Lowercase,
6. Replace newline with space,
7. Delete Repeated Char,
8. Delete single word (moved to after stopword), dan
9. Replace double space.

In [37]:
# Import Lib String
import string

In [38]:
# Before Remove Punctuation
df_clean.head(10)

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4
5,27-12-2022,TikTok - Komentar,cimory,cimoryyogurt,AyuSokowati,ayusokowati,https://www.tiktok.com/@ayusokowati,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,masyaAllah mom kerennn,3
6,27-12-2022,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Sehat sehat 🥰🥰,3
7,7-3-2023,TikTok - Komentar,cimory,cimoryyogurt,Olentia Mulier,awdidimiz4l,https://www.tiktok.com/@awdidimiz4l,0,NaN,NaN,NaN,0,https://www.tiktok.com/@yenny_eagles/video/718...,seger sehat 🥰🥰,3
8,23-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,0,https://www.tiktok.com/@yenny_eagles/video/718...,Sehat 🥰,3
9,23-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,0,https://www.tiktok.com/@yenny_eagles/video/718...,Beneran seger nih 🥰,3


In [39]:
def remove_punct(text):
    # Remove hashtag dan @
    ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

    # Change "-" to space
    text = text.replace("-", " ")

    # Remove punctiation
    text  = "".join([char for char in text if char not in string.punctuation])

    # Remove angka
    text = re.sub('[0-9]+', '', text)

    # Remove website link
    text = re.sub(r'http\S+', '', text)

    # Remove new line
    text = re.sub("\n"," ",text)

    # Transform to lowercase
    text = text.lower()

    # Replace perulangan huruf pada kalimat "iyaaa" to "iya"
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)

    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)

    # will replace space more than one
    text = re.sub('( {2,})',' ',text)

    return text

## Running
df_clean['text_punt'] = df_clean['text'].apply(lambda x: remove_punct(x))

## Show Data
df_clean.head(10)

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,text_punt
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3,belum sempetnya mau bikin 😁😁
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3,udah ikut bikin ini tadi pagi 🥰🥰
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2,cimory kesukaan keluarga nih
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3,wih pengen coba jugaa
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4,pagi ka say🥰🥰
5,27-12-2022,TikTok - Komentar,cimory,cimoryyogurt,AyuSokowati,ayusokowati,https://www.tiktok.com/@ayusokowati,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,masyaAllah mom kerennn,3,masyaallah mom keren
6,27-12-2022,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Sehat sehat 🥰🥰,3,sehat sehat 🥰🥰
7,7-3-2023,TikTok - Komentar,cimory,cimoryyogurt,Olentia Mulier,awdidimiz4l,https://www.tiktok.com/@awdidimiz4l,0,NaN,NaN,NaN,0,https://www.tiktok.com/@yenny_eagles/video/718...,seger sehat 🥰🥰,3,seger sehat 🥰🥰
8,23-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,0,https://www.tiktok.com/@yenny_eagles/video/718...,Sehat 🥰,3,sehat 🥰
9,23-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,0,https://www.tiktok.com/@yenny_eagles/video/718...,Beneran seger nih 🥰,3,beneran seger nih 🥰


####3. Penghapusan Kata Singkatan, Slang, Alay, DLL

In [40]:
import string

In [41]:
df_tmp_nasalsabila = pd.read_csv('data/slang/colloquial-indonesian-lexicon_nasalsabila.csv')
df_slang_fendiirfan = pd.read_csv('data/slang/colloquial-indonesian-lexicon_fendiirfan.csv')
df_slang_custom = pd.read_csv('data/slang/custom_dictonary_slang.csv')
df_slang_custom_exception = pd.read_csv('data/slang/custom_dictonary_slang_dont_swap.csv')

In [42]:
## Transform and Delete Duplicate Data from nasalsabila
df_slang_nasalsabila = df_tmp_nasalsabila[['slang', 'formal']]
df_slang_nasalsabila.drop_duplicates(inplace=True)

## Transform and Delete Duplicate Data from fendiirfan
df_slang_fendiirfan.dropna(inplace=True)
df_slang_fendiirfan.rename(columns={'kataAlay': 'slang', 'kataBaik': 'formal'}, inplace=True)
df_slang_fendiirfan.drop_duplicates(inplace=True)

## Transform and Delete Duplicate our own data.
df_slang_custom.drop_duplicates(inplace=True)
df_slang_custom_exception.drop_duplicates(inplace=True)

<ipython-input-42-97e6cbf1b797>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slang_nasalsabila.drop_duplicates(inplace=True)


In [43]:
############### Filter Slang awal (20 Menit)
# %%time

# def remove_slang(text):
#     # To Lowercase
#     text = " "+text.lower()+" "
#     for index, row in df_filter_slang.iterrows():
#       # Change Slang to Formal
#       # print(row['slang'], row['formal'])
#       # text = re.sub(r'\b%s\b' % row['slang'], row['formal'], text)
#       text = text.replace(' '+row['slang']+' ', ' '+row['formal']+' ')

#     text = text[1:]
#     text = text.rstrip(text[-1])
#     return text

# ## Testing
# #x = "yang yg dapat dpt blh boleh okay oke"
# #print(remove_slang(x))

# df_clean['text_filter'] = df_clean['text'].apply(lambda x: remove_slang(x))
# df_clean.head(10)

In [44]:
import numpy as np

In [45]:
%%time

def remove_slang(text, df_tmp):
    # To Lowercase
    text = text.lower()

    # Pecah to array
    words = text.split(" ");

    # DF to Array
    arr_slang = df_tmp['slang'].to_numpy()
    arr_formal = df_tmp['formal'].to_numpy()
    do_not_change = df_slang_custom_exception['kata'].to_numpy()

    # Array New Text
    new_text = []

    for kata in words:
      if kata in arr_slang:
        if kata not in do_not_change:
          idx = np.where(arr_slang==kata)[0][0]
          kata = arr_formal[idx]

      new_text.append(kata)
      text_new = ' '.join(new_text)

    return text_new

## Testing
#x = "yang yg dapat dpt blh boleh okay oke"
#print(remove_slang(x))

## Running
df_clean['text_filter1'] = df_clean['text_punt'].apply(lambda x: remove_slang(x,df_slang_nasalsabila))
df_clean['text_filter2'] = df_clean['text_filter1'].apply(lambda x: remove_slang(x,df_slang_fendiirfan))
df_clean['text_filter'] = df_clean['text_filter2'].apply(lambda x: remove_slang(x,df_slang_custom))

## Delete Unused Column
df_clean.drop(columns=['text_punt', 'text_filter1', 'text_filter2'], inplace=True)

## Show Data
df_clean

CPU times: user 31 s, sys: 889 ms, total: 31.9 s
Wall time: 31.7 s


,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,text_filter
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3,belum sempetnya mau buat 😁😁
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3,sudah ikut buat ini tadi pagi 🥰🥰
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2,cimory kesukaan keluarga ini
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3,wih pengin coba juga
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4,pagi kakak say🥰🥰
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,29-11-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,황 펠리/황 휴즈,hwang_felly,https://www.tiktok.com/@hwang_felly,0,NaN,NaN,NaN,2,https://www.tiktok.com/@tedenn/video/730678176...,protes ke ultra udh excited padahal,1,protes ke ultra sudah excited padahal
20047,15-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,youareperfect,bantufypdon,https://www.tiktok.com/@bantufypdon,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,αe`oO ✧: tostos nct aja blm dapet😭😭,1,αeoo ✧ tostos nct saja belum dapet😭😭
20048,1-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,📧,bbabiela,https://www.tiktok.com/@bbabiela,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,MKSDDD?!,1,maksud
20049,3-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,Sionah 🌟,sionahla,https://www.tiktok.com/@sionahla,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,tolong Kids,1,tolong kids


####4. Stemming Sentence

In [46]:
## Donwload Library Sastrawi for Stemming and Stopword
!pip install Sastrawi

In [47]:
# Import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Create Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [48]:
%%time

def stemming_sentence(text):
    text = stemmer.stem(text)
    return text

## Running
df_clean['text_stemming'] = df_clean['text_filter'].apply(lambda x: stemming_sentence(x))

## Delete Unused Column
df_clean.drop(columns=['text_filter'], inplace=True)

## Show Data
df_clean

CPU times: user 25min 58s, sys: 5.43 s, total: 26min 3s
Wall time: 26min 25s


,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,text_stemming
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3,belum sempetnya mau buat
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3,sudah ikut buat ini tadi pagi
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2,cimory suka keluarga ini
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3,wih pengin coba juga
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4,pagi kakak say
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,29-11-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,황 펠리/황 휴즈,hwang_felly,https://www.tiktok.com/@hwang_felly,0,NaN,NaN,NaN,2,https://www.tiktok.com/@tedenn/video/730678176...,protes ke ultra udh excited padahal,1,protes ke ultra sudah excited padahal
20047,15-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,youareperfect,bantufypdon,https://www.tiktok.com/@bantufypdon,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,αe`oO ✧: tostos nct aja blm dapet😭😭,1,eoo tostos nct saja belum dapet
20048,1-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,📧,bbabiela,https://www.tiktok.com/@bbabiela,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,MKSDDD?!,1,maksud
20049,3-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,Sionah 🌟,sionahla,https://www.tiktok.com/@sionahla,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,tolong Kids,1,tolong kids


#### 5. Remove Stopwords

In [49]:
## Load Library Stopword for Indonesian Language Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [50]:
# Load Data Stopword and Custom Word
stopword_satya = list(pd.read_csv('data/stopword/stopwords_satya.txt', header = None)[0])
custom_stopword = list(pd.read_csv('data/stopword/custom_stopword.txt', header = None)[0])
keep_custom_stopword = list(pd.read_csv('data/stopword/keep_custom_stopword.txt', header = None)[0])

In [51]:
def remove_stopwords(text):
  # Removal Stopword
  text_split = text.split(' ')

  temp_text_split = []
  for i in range(len(text_split)):
    ## Remove Stopword by Satwa : gak dipake dulu ini terlalu ngilangin konteks text
    #if (text_split[i] not in stopword_satya) and (text_split[i] not in custom_stopword):
    if (text_split[i] not in custom_stopword):
      ## Keep Beberapa Kata Valence of a text
      # https://datascience.stackexchange.com/questions/15765/nlp-why-is-not-a-stop-word#:~:text=Stop%20words%20are%20usually%20thought,idf%20analysis%20for%20document%20classification).
      if (text_split[i] in keep_custom_stopword) :
        temp_text_split.append(text_split[i])
      else :
        ## Stopword Removal from data by Sastrawi
        tmp = stopword.remove(text_split[i])
        temp_text_split.append(tmp)

  final_text = ' '.join(temp_text_split)

  # will replace single word
  final_text = re.sub(r"\b[a-zA-Z]\b","",final_text)

  # will replace space more than one
  final_text = re.sub("( {2,})"," ",final_text)

  return final_text

## Running
df_clean['text_final'] = df_clean['text_stemming'].apply(lambda x: remove_stopwords(x))

## Delete Unused Column
df_clean.drop(columns=['text_stemming'], inplace=True)

## Show Data
df_clean

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,text_final
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3,sempetnya mau buat
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3,ikut buat tadi pagi
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2,cimory suka keluarga
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3,pengin coba
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4,pagi kakak say
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,29-11-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,황 펠리/황 휴즈,hwang_felly,https://www.tiktok.com/@hwang_felly,0,NaN,NaN,NaN,2,https://www.tiktok.com/@tedenn/video/730678176...,protes ke ultra udh excited padahal,1,protes ultra excited padahal
20047,15-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,youareperfect,bantufypdon,https://www.tiktok.com/@bantufypdon,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,αe`oO ✧: tostos nct aja blm dapet😭😭,1,eoo tostos nct dapet
20048,1-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,📧,bbabiela,https://www.tiktok.com/@bbabiela,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,MKSDDD?!,1,maksud
20049,3-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,Sionah 🌟,sionahla,https://www.tiktok.com/@sionahla,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,tolong Kids,1,kids


####6. Tokenization (Commented : Belum kepake di proses sampai saat ini)

In [52]:
# df_clean
# df_clean.to_csv('data_scraped/data_ok.csv')
# df_clean.to_excel('data_scraped/data_ok.xlsx')
# df_clean

In [53]:
df_clean

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,text_final
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3,sempetnya mau buat
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3,ikut buat tadi pagi
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2,cimory suka keluarga
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3,pengin coba
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4,pagi kakak say
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,29-11-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,황 펠리/황 휴즈,hwang_felly,https://www.tiktok.com/@hwang_felly,0,NaN,NaN,NaN,2,https://www.tiktok.com/@tedenn/video/730678176...,protes ke ultra udh excited padahal,1,protes ultra excited padahal
20047,15-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,youareperfect,bantufypdon,https://www.tiktok.com/@bantufypdon,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,αe`oO ✧: tostos nct aja blm dapet😭😭,1,eoo tostos nct dapet
20048,1-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,📧,bbabiela,https://www.tiktok.com/@bbabiela,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,MKSDDD?!,1,maksud
20049,3-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,Sionah 🌟,sionahla,https://www.tiktok.com/@sionahla,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,tolong Kids,1,kids


In [54]:
# Download Library NLTK untuk Tokenizernya.
!pip install nltk

# Import Lib NLTK dan Tokenizer
import nltk
nltk.download("popular")
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [55]:
# def tokenization(text):
#     word_token = word_tokenize(text)
#     return word_token

# df_clean['text_token'] = df_clean['text_stemming'].apply(lambda x: tokenization(x.lower()))
# df_clean.head(10)

#### [LAST] Save Data Checkpoint CSV

In [59]:
## Duplicate Dataframe
df_final = df_clean

## Delete NULL value in data after preprocessing
# Replace Data null to nan
df_final.replace(' ', np.nan, inplace=True)
df_final.replace('', np.nan, inplace=True)

# Delete Data nan
df_final.dropna(subset=['text_final'], inplace=True)

## Preview Data
df_final

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,text_final
0,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,PawonDiya,pawondiya,https://www.tiktok.com/@pawondiya,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Belum sempetnya mau bikin 😁😁,3,sempetnya mau buat
1,5-1-2023,TikTok - Komentar,cimory,cimoryyogurt,Yenny eagles,olentia_mulier,https://www.tiktok.com/@olentia_mulier,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,Udah ikut bikin ini tadi pagi 🥰🥰,3,ikut buat tadi pagi
2,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,titamarifatun.nh,titamarifatun.nh,https://www.tiktok.com/@titamarifatun.nh,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,cimory kesukaan keluarga nih,2,cimory suka keluarga
3,1-1-2023,TikTok - Komentar,cimory,cimoryyogurt,TEMPATNYA REVIEW 🌻,tempatnyareview,https://www.tiktok.com/@tempatnyareview,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,wihhh pengen coba jugaa,3,pengin coba
4,29-12-2022,TikTok - Komentar,cimory,cimoryyogurt,uwie♊🐒,dwivavino,https://www.tiktok.com/@dwivavino,0,NaN,NaN,NaN,1,https://www.tiktok.com/@yenny_eagles/video/718...,pagi ka say🥰🥰,4,pagi kakak say
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,29-11-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,황 펠리/황 휴즈,hwang_felly,https://www.tiktok.com/@hwang_felly,0,NaN,NaN,NaN,2,https://www.tiktok.com/@tedenn/video/730678176...,protes ke ultra udh excited padahal,1,protes ultra excited padahal
20047,15-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,youareperfect,bantufypdon,https://www.tiktok.com/@bantufypdon,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,αe`oO ✧: tostos nct aja blm dapet😭😭,1,eoo tostos nct dapet
20048,1-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,📧,bbabiela,https://www.tiktok.com/@bbabiela,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,MKSDDD?!,1,maksud
20049,3-12-2023,TikTok - Komentar,ultramilk,ultramilkxstraykids,Sionah 🌟,sionahla,https://www.tiktok.com/@sionahla,0,NaN,NaN,NaN,0,https://www.tiktok.com/@tedenn/video/730678176...,tolong Kids,1,kids


In [60]:
platform = 'tiktok'

# Check Path is Available if no Create Dir
if path.exists(root_file+'/data/preprocess/'+platform) == False:
  os.makedirs(root_file+'/data/preprocess/'+platform)
else :
  print("File Already Existed")

# Export the Data
df_final.to_csv('data/preprocess/'+platform+'.csv', encoding='utf-8', index=False)
df_final.to_excel('data/preprocess/'+platform+'.xlsx', encoding='utf-8', index=False)

File Already Existed


/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
